In [16]:
import pandas as pd
import numpy as np
import yfinance as yf
import requests
import zipfile
import io
import xlsxwriter
import time

In [160]:
#Funções:

###Função requisicao dados CVN
# Função para fazer a requisição e tratar os possíveis erros
def fazer_requisicao(url):
    try:
        # Realizar a requisição
        response = requests.get(url)

        # Verificar o status code da resposta
        if response.status_code == 200:
            print('Requisição bem-sucedida para '+mes+'/'+ano+'.')
            return response
        elif response.status_code == 404:
            print('Página não encontrada (erro 404). Verifique a devida publicação na CVM disponível no site https://dados.cvm.gov.br/dados/FI/DOC/')
            return  response # Interrompe o código aqui se o status code for 404
        else:
            print(f'Erro inesperado: Status code {response.status_code}')
            return  # Interrompe o código em caso de status code diferente de 200 e 404

    except requests.ConnectionError:
        print('Erro de conexão: Verifique sua conexão de internet.')
    except requests.Timeout:
        print('Erro de timeout: A requisição excedeu o tempo limite.')
    except requests.RequestException as e:
        print(f'Erro na requisição: {e}')


def coleta(objetivo, mes, ano):
    
    """
    Esta função captura dados do site da CVM.

    Parâmetros:
    objetivo(string): qual o tipo de informação deseja 'lamina' ou 'informativo'.
    mes(int): qual o mês do dado.
    ano(int): qual o ano do dado.
    
    Retorno:

    df com o dado CVM 
    
    
    """
    objetivo = objetivo
   
    ano = str(ano)
    mes = str(mes).zfill(2) 
    informativo_dia  = 'https://dados.cvm.gov.br/dados/FI/DOC/INF_DIARIO/DADOS/inf_diario_fi_'+ano+mes+'.zip'
    lamina =  'https://dados.cvm.gov.br/dados/FI/DOC/LAMINA/DADOS/lamina_fi_'+ano+mes+'.zip'
    arquivo_inf_diario = 'inf_diario_fi_'+ano+mes+'.csv'
    arquivo_lamina = 'lamina_fi_'+ano+mes+'.csv'
    

    if objetivo =='informativo':
        link_arq = informativo_dia
        tp_arq = arquivo_inf_diario
    elif objetivo =='lamina':
        tp_arq = arquivo_lamina
        link_arq = lamina

    print('{}\n{}'.format(link_arq, tp_arq))
    r = fazer_requisicao(link_arq)
    
    if r.status_code == 404:
        print()
        
    else:

        zf = zipfile.ZipFile(io.BytesIO(r.content))

        arquivo = tp_arq

        zf = zf.open(arquivo)

        lines = zf.readlines()
        lines =[i.strip().decode('ISO-8859-1') for i in lines]
        lines = [i.split(';') for i in lines]

        df = pd.DataFrame(lines[1:], columns = lines[0])
        return df


In [161]:
coleta('lamina',2,2024)

https://dados.cvm.gov.br/dados/FI/DOC/LAMINA/DADOS/lamina_fi_202402.zip
lamina_fi_202402.csv
Requisição bem-sucedida para 02/2024.


,CNPJ_FUNDO,DENOM_SOCIAL,DT_COMPTC,NM_FANTASIA,ENDER_ELETRONICO,PUBLICO_ALVO,RESTR_INVEST,OBJETIVO,POLIT_INVEST,PR_PL_ATIVO_EXTERIOR,...,VL_DESPESA_3ANO,VL_DESPESA_5ANO,VL_RETORNO_3ANO,VL_RETORNO_5ANO,REMUN_DISTRIB,DISTRIB_GESTOR_UNICO,CONFLITO_VENDA,TEL_SAC,ENDER_ELETRONICO_RECLAMACAO,INF_SAC
0,00.068.305/0001-35,FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE IN...,2024-02-29,FIC EMPREENDER RF LP,www.caixa.gov.br,Investidores que buscam retorno por meio de in...,o FUNDO destina-se a acolher investimentos de ...,Proporcionar rentabilidade por meio da aplicaç...,Investir no mínimo 95% do patrimônio líquido d...,0.000000,...,52.740000,97.280000,278.260000,513.230000,O serviço de distribuição de cotas de fundos d...,O distribuidor oferta ao cotista e potenciais ...,Não se aplica,8007260101,www.caixa.gov.br,
1,00.071.477/0001-68,BB RENDA FIXA AUTOMÁTICO EMPRESA SIMPLES FUNDO...,2024-02-29,BB Renda Fixa Automático Empresa Simples,www.bb.com.br,O fundo ï¿½ destinado a investidores que prete...,,O FUNDO tem como objetivo proporcionar a renta...,"Para alcançar seus objetivos, o FUNDO aplicará...",0.000000,...,0.180000,0.340000,330.820000,610.170000,O Distribuidor fará jus a uma importância corr...,O principal Distribuidor do fundo é o Banco do...,"O Administrador, o Gestor ou partes a eles rel...",8007293886,"bbdtvm@bb.com.br - Praça XV de Novembro 20, 3º...",
2,00.073.041/0001-08,BB BESC RENDA FIXA PRÁTICO CRÉDITO PRIVADO FUN...,2024-02-29,BB Besc Renda Fixa Pratico Credito Privado,www.bb.com.br/www.besc.com.br,O fundo ï¿½ destinado a investidores que prete...,,Acompanhar o CDI mediante aplicação em ativos ...,As aplicações do FUNDO deverão se subordinar a...,0.000000,...,7.080000,13.020000,323.920000,597.490000,O Distribuidor fará jus a uma importância corr...,O principal Distribuidor do fundo é o Banco do...,"O Administrador, o Gestor ou partes a eles rel...",8007293886,"bbdtvm@bb.com.br - Praça XV de Novembro 20, 3º...",
3,00.089.915/0001-15,AMARIL FRANKLIN FUNDO DE INVEST EM COTAS DE FU...,2024-02-29,FUNDO FRANKLIN,https://bancomercantil.com.br/BeneficiarioINSS...,Destinado a investidores pessoas físicas e jur...,Voltado apenas para aplicações de investidores...,objetivo buscar proporcionar aos seus cotistas...,As aplicações do FUNDO deverão estar represent...,20.000000,...,53.860000,98.960000,103.770000,177.330000,A distribuição de cotas dos fundos administrad...,O Banco Mercantil S.A. oferta ao público alvo ...,Os fundos geridos e administrados pela Mercant...,3130576568,mercantil.distribuidora@mercantil.com.br,
4,00.180.995/0001-10,SAFRA EXECUTIVE MAX RF FUNDO DE INVESTIMENTO E...,2024-02-29,SAFRA EXECUTIVE MAX RENDA FIXA FUNDO DE INVEST...,www.safraasset.com.br,Destinado a investidores em geral que buscam u...,N/D,Buscar um retorno adicional por meio de exposi...,Aplica em fundos de investimento com carteira ...,20.000000,...,8.070000,9.770000,1322.930000,1600.740000,"A remuneração dos distribuidores, incluindo o ...","A distribuição é realizada, principalmente, pe...",Vide acima.,8007725755,www.safraasset.com.br,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5515,96.498.654/0001-66,BRADESCO FUNDO DE INVESTIMENTO EM AÇÕES MULTI ...,2024-02-29,BRADESCO FUNDO DE INVESTIMENTO EM AÇÕES MULTI ...,wwwbradescocombr,O Fundo destina-se ao público em geral que sej...,-,O objetivo do Fundo é o de proporcionar a seus...,O Fundo pretende atingir seu objetivo investin...,10.000000,...,133.550000,246.330000,197.450000,364.180000,A distribuição de cotas é remunerada através d...,Principal DistribuidorO principal distribuidor...,Regras de Conduta - Conflito de InteressesO Br...,30038330,,
5516,96.498.985/0001-04,BRADESCO FUNDO DE INVESTIMENTO EM COTAS DE FUN...,2024-02-29,BRADESCO FUNDO DE INVESTIMENTO EM COTAS DE FUN...,wwwbradescocombr,O FUNDO destina-se ao público em geral suscetí...,-,O FUNDO tem por objetivo proporcionar aos seus...,O Fundo pretende atingir seu objetivo investin...,10.000000,...,44.